<a href="https://colab.research.google.com/github/AmbrogioMB/AlgOpt/blob/main/Introduction_to_Integer_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. A first problem
Suppose we wish to invest $\$19,000$. We have identified four investment opportunities. Investment 1 requires an investment of $\$6,700$ and has a net present value of $\$8,000$; investment 2 requires $\$10,000$ and has a value of $\$11,000$; investment 3 requires $\$5,500$ and has a value of $\$6,000$; and investment 4 requires $\$3,400$ and has a value of $\$4,000$. Into which investments should we place our money so as to maximize our total present value? Each project is a ''take it or leave it'' opportunity: It is not allowed to invest partially in any of the projects. Such problems are called capital budgeting problems. As in linear programming, our first step is to decide on the variables. In this case, it is easy: We will use a $0-1$ variable $x_j$ for each investment. If $x_j$ is $1$ then we will make investment $j$. If it is $0$, we will not make the investment. This leads to the $0-1$ programming problem

In [ ]:
! pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

investments = ['Investment 1', 'Investment 2', 'Investment 3', 'Investment 4']
costs = [6.7, 10, 5.5, 3.4]
values = [8, 11, 6, 4]


model_continuous = gp.Model("example LP")

x = model_continuous.addVars(investments, name="x", vtype=GRB.CONTINUOUS, ub=1)

model_continuous.addLConstr(gp.quicksum(costs[i]*x[investments[i]] for i in range(len(investments))) <= 19, "Constraint1")

model_continuous.setObjective(gp.quicksum(values[i]*x[investments[i]] for i in range(len(investments))), GRB.MAXIMIZE)

model_continuous.optimize()

x_sol = {i: x[i].x for i in investments}
print("Decision variables:")
print(x_sol)

Unfortunately, this solution is not integral. Rounding $x_2$ down to $0$ gives a
feasible solution with a value of $\$12,000$.

In [ ]:
x_round = {i: int(x[i].x) for i in investments}
print("Decision variables (rounded):")
print(x_round)
cost_round = sum(costs[i]*x_round[investments[i]] for i in range(len(investments)))
value_round = sum(values[i]*x_round[investments[i]] for i in range(len(investments)))
print("Objective value rounded solution:", value_round)
print("Cost rounded solution:", cost_round)

There is a better integer solution, however.

In [ ]:
model_integer = gp.Model("example ILP")

x = model_integer.addVars(investments, name="x", vtype=GRB.BINARY)

model_integer.addLConstr(gp.quicksum(costs[i]*x[investments[i]] for i in range(len(investments))) <= 19, "Constraint1")

model_integer.setObjective(gp.quicksum(values[i]*x[investments[i]] for i in range(len(investments))), GRB.MAXIMIZE)

model_integer.optimize()
x_sol = {i: x[i].x for i in investments}
print("Decision variables:")
print(x_sol)
print("Objective value:", model_integer.objVal)

This example shows that rounding does not necessarily give an optimal solution.

There are a number of additional constraints we might want to add. For instance, consider the following constraints:
1. We can only make two investments.
2. If investment 2 is made, then investment 4 must also be made.
3. If investment 1 is made, then investment 3 cannot be made.

All of these, and many more *logical restrictions*, can be enforced using
$0-1$ variables. In these cases, the constraints are the following

In [ ]:
model_integer.addLConstr(gp.quicksum(x[investments[i]] for i in range(len(investments))) <= 2, "Constraint2")
model_integer.addLConstr(x['Investment 2'] - x['Investment 4'] <= 0, "Constraint3")
model_integer.addLConstr(x['Investment 1'] - x['Investment 3'] <= 1, "Constraint4")
model_integer.update()
model_integer.optimize()
x_sol = {i: x[i].x for i in investments}
print("Decision variables:")
print(x_sol)
print("Objective value:", model_integer.objVal)

## 2. Exercise: oil exploration
As the leader of an oil exploration drilling venture, you must determine the best selection of 5 out of 10 possible sites. Label the sites $s_1, s_2, \dots, s_{10}$ and the expected profits associated with each as $p_1, p_2, \dots, p_{10}$.

1. If site $s_2$ is explored, then site $s_3$ must also be explored. Furthermore, regional development restrictions are such that
2. Exploring sites $s_1$ and $s_7$ will prevent you from exploring site $s_8$.
3. Exploring sites $s_3$ or $s_4$ will prevent you from exploring site $s_5$.

In [ ]:
import numpy as np
m_oil = gp.Model("Oil drilling")
p = np.random.randint(1, 20, 10)
# variables and constraints here

## 3. Exercise: Sudoku

The game is played on a $9 \times 9$ grid which is subdivided into $9$ blocks of $3 \times 3$ contiguous cells. The grid must be filled with numbers $1, \dots ,9$ so that all the numbers between $1$ and $9$ appear in each row, in each column and in each of the nine blocks. A game consists of an initial assignment of numbers in some cells.

\begin{array}{|ccc|ccc|ccc|}
\hline
  8  &  - &  - &  - & 2  & 6  &  - & -  &  - \\
  -  &  - &  - &  - & -  & -  &  7 & -  &  4 \\
  -  &  - &  - &  7 & -  & -  &  - & -  &  5 \\
  \hline
  -  &  - &  - &  1 & -  & -  &  - & 3  &  6 \\
  -  &  1 &  - &  - & 8  & -  &  - & 4  &  - \\
  9  &  8 &  - &  - & -  & 3  &  - & -  &  - \\
  \hline
  3  &  - &  - &  - & -  & 1  &  - & -  &  - \\
  7  &  - &  5 &  - & -  & -  &  - & -  &  - \\
  -  &  - &  - &  2 & 5  & -  &  - & -  &  8 \\
\hline
\end{array}

This is a decision problem that can be modeled with binary variables
$x_{ijk}$, $1 \leq  i, j, k \leq 9$ where $x_{ijk} = 1$ if number $k$ is entered in position with coordinates $i, j$ of the grid, and $0$ otherwise.

In [ ]:
m_sudoku = gp.Model("Sudoku")
# variables and constraints here